In [1]:
import sys, os, subprocess, glob

# 1) Descubre el prefix de Homebrew
try:
    prefix = subprocess.check_output(["brew", "--prefix"]).decode().strip()
except Exception:
    # Fallback típico en Apple Silicon si PATH no trae brew al notebook
    prefix = "/opt/homebrew"

# 2) Busca cualquier site-packages de OLA para python3.*
candidates = glob.glob(os.path.join(prefix, "opt", "ola", "libexec", "lib", "python3.*", "site-packages"))

# 3) Añade todos los candidatos al sys.path
added = []
for p in candidates:
    if os.path.isdir(p) and p not in sys.path:
        sys.path.append(p)
        added.append(p)

print("Añadidas a sys.path:", added or "ninguna; ¿OLA está instalada?")

# 4) Importa OLA
from ola.ClientWrapper import ClientWrapper
print("OLA importada OK")

Añadidas a sys.path: ['/opt/homebrew/opt/ola/libexec/lib/python3.13/site-packages']
OLA importada OK


/opt/homebrew/opt/ola/libexec/lib/python3.13/site-packages/ola/rpc/SimpleRpcController.py:18: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service


In [8]:
from array import array
from ola.ClientWrapper import ClientWrapper

def set_channel_once(universe: int, channel_1_based: int, value_0_255: int):
    # Construye un frame de 512 con todo a 0 y solo un canal modificado
    levels = array("B", [0] * 512)
    idx = channel_1_based - 1  # DMX es 1..512
    levels[idx] = max(0, min(255, value_0_255))

    def dmx_sent(status):
        print("OK" if status.Succeeded() else "Fallo")
        wrapper.Stop()

    wrapper = ClientWrapper()
    client = wrapper.Client()
    client.SendDmx(universe, levels, dmx_sent)
    wrapper.Run()

# Ejemplo: canal 7 a 120 en el universo 1
set_channel_once(universe=1, channel_1_based=1, value_0_255=120)
wait(2)
set_channel_once(universe=1, channel_1_based=2, value_0_255=120)

OK
OK


In [7]:
from time import sleep as wait
print("Esperando 2 segundos...")

Esperando 2 segundos...
